In [1]:
import os
import random
import pickle

import comet_ml
from comet_ml import Artifact

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau

from teste import engine, utils
from common.CustomData import MRI_NII_2D
from common.data_setup import data_transforms, DOMAINS
from common.networks import (
    ARCHITECTURES,
    Classifier,
    DenseNet201WithCustomClassifierDropout,
    DenseNet169WithCustomClassifierDropout,
    DAMS,
)

from torchinfo import summary

# Setup target device
device = "cuda:5" if torch.cuda.is_available() else "cpu"
print(device)

/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


cuda:5


In [2]:
def contar_arquivos_em_subpastas(caminho):
    for pasta, subpastas, arquivos in os.walk(caminho):
        if len(arquivos) != 0:
            try:
                DOMAIN, SLIT, CLASS = rf"{pasta}".split("/")[-3:]
                print(f"{DOMAIN}/{SLIT}/{CLASS} contém {len(arquivos)} arquivos")
            except:
                print(pasta.split("/"))


# Exemplo de uso:
caminho = r"data/ADNI1-T1-AD-CN/Image/Preprocess/6_step_nifti_2d"
contar_arquivos_em_subpastas(caminho)

In [3]:
import multiprocessing

multiprocessing.cpu_count()

224

In [4]:
# Setup hyperparameters
NUM_EPOCHS = 50
NUM_WORKES = 16
SOURCE = "Siemens"
TARGET_1 = "Philips"
TARGET_2 = "GE"
EXP_NUMBER = "DAMS-SGD-8"
config = {
    # Pick the algorithm:
    "algorithm": "grid",
    # Declare your hyperparameters:
    "parameters": {
        "lr": {"type": "discrete", "values": [0.001, 0.0009, 0.0007]},  # 0.001,
        "batch_size": {"type": "discrete", "values": [16, 32, 64]},
        "dropout": {"type": "discrete", "values": [0.2, 0.3, 0.4]},
        "momentum": {
            "type": "discrete",
            "values": [0.9],
        },  # [0.0005, 0.0001, 0.005, 0.001]
        "weight_decay": {
            "type": "discrete",
            "values": [0.03, 0.02],
        },  # [0, 0.01, 0.001] - [0.0001, 0.001, 0.01]
        "model": {
            "type": "categorical",
            "values": ["DAMS"],
        },
        # "transform": {"type": "categorical", "values": ["train", "train_alt"]},
        "source": {"type": "categorical", "values": [SOURCE]},
        "target_1": {"type": "categorical", "values": [TARGET_1]},
        "target_2": {"type": "categorical", "values": [TARGET_2]},
        "seed": {"type": "discrete", "values": [42]},
    },
    # Declare what to optimize, and how:
    "spec": {
        "metric": "val_auc",
        "objective": "maximize",
    },
    "name": f"SOURCE_{SOURCE}",
    "trials": 1,
}

In [5]:
def seed_everything(seed: int):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # g = torch.Generator()
    # g.manual_seed(0)
    # return g

In [ ]:
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

# Initialize the Comet SDK
comet_ml.login(project_name=f"ADNI1-s-{SOURCE}-{EXP_NUMBER}")
opt = comet_ml.Optimizer(config=config, verbose=0)

for exp in opt.get_experiments():
    # exp.log_parameter("random_state", 42)
    print(f'Treinando esse modelo: {exp.get_parameter("model")}')
    print(
        f'Treinando com o source: {exp.get_parameter("source")}/target: {exp.get_parameter("target_1")}/{exp.get_parameter("target_2")}'
    )

    artifact = Artifact("Engine", "Code")
    artifact.add("teste/engine.py")

    exp.log_artifact(artifact)

    artifact = Artifact("Networks", "Code")
    artifact.add("common/networks.py")

    exp.log_artifact(artifact)

    artifact = Artifact("DataSetup", "Code")
    artifact.add("common/data_setup.py")

    exp.log_artifact(artifact)

    MODEL = exp.get_parameter("model")
    SOURCE = exp.get_parameter("source")
    TARGET_1 = exp.get_parameter("target_1")
    TARGET_2 = exp.get_parameter("target_2")
    seed_everything(exp.get_parameter("seed"))

    train_data_custom = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{SOURCE}"],
        split="train",
        transform=data_transforms["train_alt"],
    )
    val_data_custom = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{SOURCE}"],
        split="val",
        transform=data_transforms["val_test"],
    )
    test_data_custom = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{SOURCE}"],
        split="test",
        transform=data_transforms["val_test"],
    )

    val_data_target = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{TARGET_1}"],
        split="val",
        transform=data_transforms["val_test"],
    )
    test_data_target = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{TARGET_1}"],
        split="test",
        transform=data_transforms["val_test"],
    )

    val_data_target_2 = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{TARGET_2}"],
        split="val",
        transform=data_transforms["val_test"],
    )
    test_data_target_2 = MRI_NII_2D.Dataset2D(
        domain=DOMAINS[f"ADNI1-{TARGET_2}"],
        split="test",
        transform=data_transforms["val_test"],
    )

    train_dataloader = DataLoader(
        dataset=train_data_custom,  # use custom created train Dataset
        batch_size=exp.get_parameter("batch_size"),  # how many samples per batch?
        num_workers=NUM_WORKES,  # how many subprocesses to use for data loading? (higher = more)
        shuffle=True,
        pin_memory=True,
        # generator=g,
    )  # shuffle the data?
    val_dataloader = DataLoader(
        dataset=val_data_custom,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    test_dataloader = DataLoader(
        dataset=test_data_custom,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data

    val_target = DataLoader(
        dataset=val_data_target,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    test_target = DataLoader(
        dataset=test_data_target,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data

    val_target_2 = DataLoader(
        dataset=val_data_target_2,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    test_target_2 = DataLoader(
        dataset=test_data_target_2,  # use custom created test Dataset
        batch_size=exp.get_parameter("batch_size"),
        num_workers=NUM_WORKES,
        shuffle=False,
        pin_memory=True,
        # generator=g,
    )  # don't usually need to shuffle testing data
    print("Loading Dataloader")
    # Create model with help from model_builder.py

    if MODEL in ["DAMS"]:
        model = DAMS(dropout=exp.get_parameter("dropout")).to(device)
    elif MODEL in ["densenet201MOD"]:
        model = DenseNet201WithCustomClassifierDropout(dropout_rate=0.4).to(device)
    elif MODEL in ["densenet169MOD"]:
        model = DenseNet169WithCustomClassifierDropout(dropout_rate=0.4).to(device)
    else:
        model = ARCHITECTURES[MODEL]["model"](
            weights=ARCHITECTURES[MODEL]["weights"]
        ).to(
            device
        )  # ().to(device)#
        # Congelar a primeira metade das camadas convolucionais
        for name, param in model.named_parameters():
            if name.startswith(("conv1", "bn1", "layer1", "layer2")):
                param.requires_grad = False

    if MODEL in ["densenet121", "densenet161", "densenet169", "densenet201"]:
        num_features = model.classifier.in_features
        model.classifier = Classifier(num_features).to(device)
    elif MODEL in ["efficientnet_b0", "efficientnet_b4"]:
        num_features = model.classifier[1].in_features
        model.classifier[1] = Classifier(num_features).to(device)
    elif MODEL in ["vgg16"]:
        num_features = model.classifier[6].in_features
        model.classifier[6] = Classifier(num_features).to(device)
        for param in model.classifier.parameters():
            param.requires_grad = True  # Mantém treinável
    elif MODEL in ["DAMS"]:
        print("Use DAMS Basic")
    elif MODEL in ["densenet169MOD"]:
        print("Use densenet169MOD")
    elif MODEL in ["densenet201MOD"]:
        print("Use densenet201MOD")
    else:
        model.fc = Classifier(512, dropout=exp.get_parameter("dropout")).to(device)
    print("Create model")

    # for param in model.backbone.parameters():
    #    param.requires_grad = False
    # for param in model.backbone.fc.parameters():
    #    param.requires_grad = True
    if SOURCE == "GE":
        # pos_weight = torch.tensor([5856 / 9312]).to(device)
        # exp.log_parameter('pos_weight', 5856 / 9312)
        pos_weight = torch.tensor([6400 / 10144]).to(device)
        exp.log_parameter("pos_weight", 6400 / 10144)
    elif SOURCE == "Siemens":
        # pos_weight = torch.tensor([5856 / 9312]).to(device)
        # exp.log_parameter('pos_weight', 5856 / 9312)
        pos_weight = torch.tensor([4608 / 8000]).to(device)
        exp.log_parameter("pos_weight", 4608 / 8000)
    elif SOURCE == "Philips":
        # pos_weight = torch.tensor([2144 / 3456]).to(device)
        # exp.log_parameter('pos_weight', 2144 / 3456)
        pos_weight = torch.tensor([1792 / 2688]).to(device)
        exp.log_parameter("pos_weight", 1792 / 2688)
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=exp.get_parameter("lr"),
        weight_decay=exp.get_parameter("weight_decay"),
        momentum=0.9,
    )
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=7500, eta_min=0.004)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.2)

    # Start training with help from engine.py
    print("Start Train")
    result = engine.train(
        model=model,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        test_dataloader=test_dataloader,
        val_target=val_target,
        test_target=test_target,
        val_target_2=val_target_2,
        test_target_2=test_target_2,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=None,
        epochs=NUM_EPOCHS,
        early_stop_patience=20,
        exp=exp,
        checkpoint_path=rf"teste/models/{exp.get_name()}.pth",
        device=device,
    )

COMET INFO: a6175d5da17447a2a07d393e335bcde5
COMET INFO: Using optimizer config: {'algorithm': 'grid', 'configSpaceSize': 54, 'endTime': None, 'id': 'a6175d5da17447a2a07d393e335bcde5', 'lastUpdateTime': None, 'maxCombo': 0, 'name': 'SOURCE_Siemens', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'dropout': {'type': 'discrete', 'values': [0.2, 0.3, 0.4]}, 'lr': {'type': 'discrete', 'values': [0.001, 0.0009, 0.0007]}, 'model': {'type': 'categorical', 'values': ['DAMS']}, 'momentum': {'type': 'discrete', 'values': [0.9]}, 'seed': {'type': 'discrete', 'values': [42]}, 'source': {'type': 'categorical', 'values': ['Siemens']}, 'target_1': {'type': 'categorical', 'values': ['Philips']}, 'target_2': {'type': 'categorical', 'values': ['GE']}, 'weight_decay': {'type': 'discrete', 'values': [0.03, 0.02]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'val_auc', 'minSampleSize': 100, 'objective': 'maximize', 'randomize': False, 'retryAssignLimit'

Treinando esse modelo: DAMS
Treinando com o source: Siemens/target: Philips/GE


COMET INFO: Artifact 'Engine' version 77.0.0 created (previous was: 76.0.0)
COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 8.21 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/Engine:77.0.0' has started uploading asynchronously
COMET INFO: Artifact 'Networks' version 77.0.0 created (previous was: 76.0.0)
COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 5.82 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/Networks:77.0.0' has started uploading asynchronously
COMET INFO: Artifact 'DataSetup' version 79.0.0 created (previous was: 78.0.0)
COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 7.72 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/DataSetup:79.0.0' has started uploading asynchronously


Loading Dataloader
Use DAMS Basic
Create model
Start Train


  0%|          | 0/50 [00:00<?, ?it/s]

...

Treinando esse modelo: DAMS
Treinando com o source: Siemens/target: Philips/GE


COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 8.21 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/Engine:126.0.0' has started uploading asynchronously
COMET INFO: Artifact 'Networks' version 126.0.0 created (previous was: 125.0.0)
COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 5.82 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/Networks:126.0.0' has started uploading asynchronously
COMET INFO: Artifact 'DataSetup' version 128.0.0 created (previous was: 127.0.0)
COMET INFO: Scheduling the upload of 1 assets: 1 local assets for a size of 7.72 KB, and 0 remote assets (will be linked, not uploaded). This can take some time.
COMET INFO: Artifact 'derick-amontagna/DataSetup:128.0.0' has started uploading asynchronously


Loading Dataloader
Use DAMS Basic
Create model
Start Train


  0%|          | 0/50 [00:00<?, ?it/s]

/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
COMET INFO: Artifact 'derick-amontagna/Networks:126.0.0' has been fully uploaded successfully
COMET INFO: Artifact 'derick-amontagna/DataSetup:128.0.0' has been fully uploaded successfully
COMET INFO: Artifact 'derick-amontagna/Engine:126.0.0' has been fully uploaded successfully
/root/anaconda3/envs/lapixdl/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
